# Scraping

In [5]:
from lxml import html  
import requests
import pandas as pd
import time
import pickle
import re

product_id = 'B07C9GHXFW'

In [6]:
from os import path
if path.exists(product_id+'.pickle'):
    pass
else:
    amazon_url = 'https://www.amazon.in/product-reviews/'+ product_id + '?pageNumber=1'
    user_agent = 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36'
    headers = {'User-Agent': user_agent}
    page = requests.get(amazon_url, headers = headers)
    parser = html.fromstring(page.content)

    total_reviews = 0

    for el in parser.xpath('//div[@data-hook="cr-filter-info-section"]/span[@data-hook="cr-filter-info-review-count"]//text()'):
        total_reviews = int(re.search(r'\d+', el[14:]).group())

    #max iterations now are 5-7
    max_page_no=30
    if total_reviews < 50:
        #exception = 1
        pass
    elif total_reviews > 70:
        max_page_no = 7
    else:
        max_page_no = 30  #total_reviews//10

    for j in range(1,max_page_no):
        try:
            time.sleep(1)
            amazon_url = 'https://www.amazon.in/product-reviews/'+ product_id + '?pageNumber=' + str(j)
            print(amazon_url)
            user_agent = 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36'
            if(j==1):
                reviews_df = pd.DataFrame()

            headers = {'User-Agent': user_agent}
            page = requests.get(amazon_url, headers = headers)
            parser = html.fromstring(page.content)

            xpath_reviews = '//div[@data-hook="review"]'
            reviews = parser.xpath(xpath_reviews)

            xpath_rating  = './/i[@data-hook="review-star-rating"]//text()' 
            xpath_title   = './/a[@data-hook="review-title"]//text()'
            xpath_author  = './/a[@data-hook="review-author"]//text()'
            xpath_date    = './/span[@data-hook="review-date"]//text()'
            xpath_body    = './/span[@data-hook="review-body"]//text()'
            xpath_helpful = './/span[@data-hook="helpful-vote-statement"]//text()'

            for review in reviews:
                rating  = review.xpath(xpath_rating)
                title   = review.xpath(xpath_title)
                author  = review.xpath(xpath_author)
                date    = review.xpath(xpath_date)
                body    = review.xpath(xpath_body)
                helpful = review.xpath(xpath_helpful)

                review_dict = {'rating': rating,
                               'title': title,
                               'author': author,             
                               'date': date,
                               'body': body,
                               'helpful': helpful}
                reviews_df = reviews_df.append(review_dict, ignore_index=True)
        except requests.exceptions.ConnectionError:
            print("Connection refused")
    reviews_df.to_pickle(product_id+'.pickle')

https://www.amazon.in/product-reviews/B07C9GHXFW?pageNumber=1
https://www.amazon.in/product-reviews/B07C9GHXFW?pageNumber=2
https://www.amazon.in/product-reviews/B07C9GHXFW?pageNumber=3
https://www.amazon.in/product-reviews/B07C9GHXFW?pageNumber=4
https://www.amazon.in/product-reviews/B07C9GHXFW?pageNumber=5
https://www.amazon.in/product-reviews/B07C9GHXFW?pageNumber=6
https://www.amazon.in/product-reviews/B07C9GHXFW?pageNumber=7
https://www.amazon.in/product-reviews/B07C9GHXFW?pageNumber=8
https://www.amazon.in/product-reviews/B07C9GHXFW?pageNumber=9
https://www.amazon.in/product-reviews/B07C9GHXFW?pageNumber=10
https://www.amazon.in/product-reviews/B07C9GHXFW?pageNumber=11
https://www.amazon.in/product-reviews/B07C9GHXFW?pageNumber=12
https://www.amazon.in/product-reviews/B07C9GHXFW?pageNumber=13
https://www.amazon.in/product-reviews/B07C9GHXFW?pageNumber=14
https://www.amazon.in/product-reviews/B07C9GHXFW?pageNumber=15
https://www.amazon.in/product-reviews/B07C9GHXFW?pageNumber=16
h

In [124]:
reviews_df = pd.DataFrame()
reviews_df = pd.read_pickle(product_id+'.pickle')
reviews_df.head()

author                                               body  \
0     []  [Read my Review,  earphones i own, sennisher c...   
1     []  [I Bought this Earphone on Jun 25 After one mo...   
2     []  [Hey you... you a budget buyer? good thing you...   
3     []  [Absolutely cool product considering the price...   
4     []  [Nit a good product! Mine stopped working with...   

                date                          helpful                rating  \
0  [26 October 2018]  [155 people found this helpful]  [5.0 out of 5 stars]   
1    [7 August 2018]  [262 people found this helpful]  [1.0 out of 5 stars]   
2  [22 October 2018]  [121 people found this helpful]  [4.0 out of 5 stars]   
3  [5 December 2018]   [42 people found this helpful]  [5.0 out of 5 stars]   
4   [28 August 2018]  [148 people found this helpful]  [1.0 out of 5 stars]   

                                               title  
0                       [Bang for Bucks, \n        ]  
1  [One Side will Stop Working think before buyin...  
2  [More than what you think, More than what you ...  
3  [Tips for long lasting use of this Amazing dev...  
4                        [Dont buy them, \n        ]

_

# Combining Reviews

In [147]:
raw_reviews_list = []
raw_bulk_review = ""
for review in (reviews_df["title"] + reviews_df["body"]):
    raw_reviews_list.append(" ".join(review))
raw_bulk_review = " ".join(raw_reviews_list)
raw_bulk_review = raw_bulk_review.replace(u"’", u"'")
raw_bulk_review[:2000]

"Bang for Bucks \n         Read my Review  earphones i own sennisher cx 180 Mi in-ear Basic 1 More Piston fit Sony xb-55 and am a audiophile i like vocals and deep bass at the same time Cx 180 is the worst (i don't know why people  are mad about cx 180 but after using Mi in ear basic u will throw sennisher cx 180 i promise that) sony xb55 has nothing but bass ( no vocals and treble) 1 more piston is the best under Rs. 1000 earphone (give it a time 20-30 days driver burn time) now lets talk about Mi in ear basic earphone i would say mi in ear is litter brother of  1more piston fit.  its sound quality is superb . it has  deep bass  and good treble. i would say  its all rounder. mic quality is superb  bro i recorded audio from its mic and was blown away .they are providing  best sound and mic quality in under 400 earphone . build quality is like you are holding 1000 rupees earphone ever better than piston fit . it has aluminum casing  . no brand is providing aluminum casing under 400 rupe

In [148]:
raw_bulk_review_lower = raw_bulk_review.lower()
raw_bulk_review_lower[:2000]

"bang for bucks \n         read my review  earphones i own sennisher cx 180 mi in-ear basic 1 more piston fit sony xb-55 and am a audiophile i like vocals and deep bass at the same time cx 180 is the worst (i don't know why people  are mad about cx 180 but after using mi in ear basic u will throw sennisher cx 180 i promise that) sony xb55 has nothing but bass ( no vocals and treble) 1 more piston is the best under rs. 1000 earphone (give it a time 20-30 days driver burn time) now lets talk about mi in ear basic earphone i would say mi in ear is litter brother of  1more piston fit.  its sound quality is superb . it has  deep bass  and good treble. i would say  its all rounder. mic quality is superb  bro i recorded audio from its mic and was blown away .they are providing  best sound and mic quality in under 400 earphone . build quality is like you are holding 1000 rupees earphone ever better than piston fit . it has aluminum casing  . no brand is providing aluminum casing under 400 rupe

# Cleaning

In [149]:
contractions = { "ain't":"are not","aren't":"are not","can't":"cannot","can't've":"cannot have",
                "'cause":"because","could've":"could have","couldn't":"could not","couldn't've":"could not have",
                "didn't":"did not","doesn't":"does not","don't":"do not","hadn't":"had not",
                "hadn't've":"had not have","hasn't":"has not","haven't":"have not","he'd":"he would",
                "he'd've":"he would have","he'll":"he will","he'll've":"he will have","he's":"he has",
                "how'd":"how did","how'd'y":"how do you","how'll":"how will","how's":"how is",
                "i'd":"i would","i'd've":"i would have","i'll":"i will","i'll've":"i will have","i'm":"i am",
                "i've":"i have","isn't":"is not","it'd":"it would","it'd've":"it would have","it'll":"it will",
                "it'll've":"it will have","it's":"it is","let's":"let us","ma'am":"madam","mayn't":"may not",
                "might've":"might have","mightn't":"might not","mightn't've":"might not have",
                "must've":"must have","mustn't":"must not","mustn't've":"must not have","needn't":"need not",
                "needn't've":"need not have","o'clock":"of the clock","oughtn't":"ought not",
                "oughtn't've":"ought not have","shan't":"shall not","sha'n't":"shall not",
                "shan't've":"shall not have","she'd":"she would","she'd've":"she would have","she'll":"she will",
                "she'll've":"she will have","she's":"she is","should've":"should have","shouldn't":"should not",
                "shouldn't've":"should not have","so've":"so have","so's":"so is","that'd":"that would",
                "that'd've":"that would have","that's":"that is","there'd":"there would",
                "there'd've":"there would have","there's":"there is","they'd":"they would","they'd've":"they would have",
                "they'll":"they will","they'll've":"they will have","they're":"they are","they've":"they have",
                "to've":"to have","wasn't":"was not","we'd":"we would","we'd've":"we would have","we'll":"we will",
                "we'll've":"we will have","we're":"we are","we've":"we have","weren't":"were not","what'll":"what will",
                "what'll've":"what will have","what're":"what are","what's":"what is","what've":"what have",
                "when's":"when is","when've":"when have","where'd":"where did","where's":"where is",
                "where've":"where have","who'll":"who will","who'll've":"who will have","who's":"who is",
                "who've":"who have","why's":"why is","why've":"why have","will've":"will have","won't":"will not",
                "won't've":"will not have","would've":"would have","wouldn't":"would not","wouldn't've":"would not have",
                "y'all":"you all","y'all'd":"you all would","y'all'd've":"you all would have","y'all're":"you all are",
                "y'all've":"you all have","you'd":"you would","you'd've":"you would have","you'll":"you will",
                "you'll've":"you will have","you're":"you are","you've":"you have"}
slangs = {"coz":"because","cos":"because","plz":"please"}

In [150]:
bulk_review_lower_clean = ""
for word in raw_bulk_review_lower.split(" "):
    if word in contractions.keys():
        bulk_review_lower_clean += contractions[word] + " " 
    elif word in slangs.keys():
        bulk_review_lower_clean += slangs[word] + " "
    else:
        bulk_review_lower_clean += word + " " 
        
bulk_review_lower_clean[:2000]

"bang for bucks \n         read my review  earphones i own sennisher cx 180 mi in-ear basic 1 more piston fit sony xb-55 and am a audiophile i like vocals and deep bass at the same time cx 180 is the worst (i do not know why people  are mad about cx 180 but after using mi in ear basic u will throw sennisher cx 180 i promise that) sony xb55 has nothing but bass ( no vocals and treble) 1 more piston is the best under rs. 1000 earphone (give it a time 20-30 days driver burn time) now lets talk about mi in ear basic earphone i would say mi in ear is litter brother of  1more piston fit.  its sound quality is superb . it has  deep bass  and good treble. i would say  its all rounder. mic quality is superb  bro i recorded audio from its mic and was blown away .they are providing  best sound and mic quality in under 400 earphone . build quality is like you are holding 1000 rupees earphone ever better than piston fit . it has aluminum casing  . no brand is providing aluminum casing under 400 rup

## Stop words removal

In [151]:
stopwords = {'this','the','such', "aren't", 'few', 'hasn', 'won', 'was', 'after', 'who', 'where', 'me', 've', 'haven', "you'll", 'other', 'having', "weren't", 'has', 'of', "doesn't", 'during', 'above', 'him', 'yourselves', 'hers', 'd', 'both', 'then', 'do', 'down', 'is', 'there', 'these', 'why', 'when', 'through', 'here', 'but', 'those', "you'd", 'being', "wasn't", 'about', 'yourself', 'your', 'what', 'any', 'more', 'mightn', "wouldn't", 'which', 'himself', 'further', 'm', 'hadn', 'can', 'this', 'and', 'my', 'only', 'ours', 'too', 'shan', 'their', 'for', 'now', 'o', "mustn't", 'he', 'couldn', 'you', 'not', 'itself', 'our', "haven't", 'ain', 'themselves', "couldn't", "should've", "needn't", 'isn', 'shouldn', 'if', 'wouldn', "don't", 'she', 'wasn', 'than', 'from', 'just', 'will', 'mustn', 'on', "hasn't", 'an', 'be', 'up', 'theirs', "shouldn't", 'myself', 'does', "she's", 'should', 'ourselves', 's', 'between', 'into', 'all', 'yours', 'as', 'his', "mightn't", 'were', 'against', 'to', 'with', "isn't", 'its', 'or', "you've", 'i', 'it', 'very', 'y', 'they', "that'll", 'have', 'had', 'the', 'while', 'nor', 'herself', 'out', 'how', 're', 'below', 'under', "you're", 'in', 'own', 'same', 'll', 'each', 'ma', "didn't", 't', 'that', 'a', 'so', 'needn', 'because', 'once', 'been', 'did', 'before', 'are', 'aren', 'am', 'again', 'doesn', 'until', 'no', 'don', "shan't", 'her', 'some', 'over', 'off', 'weren', 'doing', 'by', 'most', "won't", 'we', "it's", 'at', 'them', 'didn', 'whom', "hadn't"}

stop_words = set(stopwords)
#print(stop_words)
list_words = [' '.join(w for w in place.split() if w.lower() not in stop_words)
         for place in bulk_review_lower_clean.split(" ")
         ]
bulk_review_lower_wo_sw = " ".join(list_words)

In [152]:
# TODO: removing extra spaces
import re

bulk_review_lower_wo_sw= re.sub("\s\s+" , " ", bulk_review_lower_wo_sw)
bulk_review_lower_wo_sw= re.sub("[\.][\.]+" , ".", bulk_review_lower_wo_sw) 
bulk_review_lower_wo_sw= re.sub("[\!]+" , ".", bulk_review_lower_wo_sw)
bulk_review_lower_wo_sw[:2000]

"bang bucks read review earphones sennisher cx 180 mi in-ear basic 1 piston fit sony xb-55 audiophile like vocals deep bass time cx 180 worst (i know people mad cx 180 using mi ear basic u throw sennisher cx 180 promise that) sony xb55 nothing bass ( vocals treble) 1 piston best rs. 1000 earphone (give time 20-30 days driver burn time) lets talk mi ear basic earphone would say mi ear litter brother 1more piston fit. sound quality superb . deep bass good treble. would say rounder. mic quality superb bro recorded audio mic blown away .they providing best sound mic quality 400 earphone . build quality like holding 1000 rupees earphone ever better piston fit . aluminum casing . brand providing aluminum casing 400 rupees earphone . u youtuber mic replace mic price range 1000 .it good mic trust cons :- cons considering price mic mi piston great .but does'nt bother high budget go mi 1 piston fit .it everything even sounds louder mi ear basic one side stop working think buying bought earphone 

In [153]:
sentences = bulk_review_lower_wo_sw.split('.')
sentences[:50]

['bang bucks read review earphones sennisher cx 180 mi in-ear basic 1 piston fit sony xb-55 audiophile like vocals deep bass time cx 180 worst (i know people mad cx 180 using mi ear basic u throw sennisher cx 180 promise that) sony xb55 nothing bass ( vocals treble) 1 piston best rs',
 ' 1000 earphone (give time 20-30 days driver burn time) lets talk mi ear basic earphone would say mi ear litter brother 1more piston fit',
 ' sound quality superb ',
 ' deep bass good treble',
 ' would say rounder',
 ' mic quality superb bro recorded audio mic blown away ',
 'they providing best sound mic quality 400 earphone ',
 ' build quality like holding 1000 rupees earphone ever better piston fit ',
 ' aluminum casing ',
 ' brand providing aluminum casing 400 rupees earphone ',
 ' u youtuber mic replace mic price range 1000 ',
 'it good mic trust cons :- cons considering price mic mi piston great ',
 "but does'nt bother high budget go mi 1 piston fit ",
 'it everything even sounds louder mi ear basi

In [154]:
stuff = []
stuff.append(bulk_review_lower_wo_sw)
stuff

['bang bucks read review earphones sennisher cx 180 mi in-ear basic 1 piston fit sony xb-55 audiophile like vocals deep bass time cx 180 worst (i know people mad cx 180 using mi ear basic u throw sennisher cx 180 promise that) sony xb55 nothing bass ( vocals treble) 1 piston best rs. 1000 earphone (give time 20-30 days driver burn time) lets talk mi ear basic earphone would say mi ear litter brother 1more piston fit. sound quality superb . deep bass good treble. would say rounder. mic quality superb bro recorded audio mic blown away .they providing best sound mic quality 400 earphone . build quality like holding 1000 rupees earphone ever better piston fit . aluminum casing . brand providing aluminum casing 400 rupees earphone . u youtuber mic replace mic price range 1000 .it good mic trust cons :- cons considering price mic mi piston great .but does\'nt bother high budget go mi 1 piston fit .it everything even sounds louder mi ear basic one side stop working think buying bought earphon

## TFIDF (yash)

In [155]:
import nltk
import re
input_text = re.sub(r'[^\w\s]','',raw_bulk_review)
from nltk.tokenize import sent_tokenize, word_tokenize
sentence = sent_tokenize(input_text)
review_subtext = input_text.split("8888888")

from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
tfidf_input=""
for review in review_subtext:
    temp_review = review.split(" ")
    filtered_words = [w for w in temp_review if not w in stop_words]
    filtered_words = list(filter(None, filtered_words))
    temp= nltk.pos_tag(filtered_words)
    for k in temp:
        if(k[1]=='NN' or k[1]=='NNS'):
            tfidf_input = tfidf_input + k[0] + " "
    tfidf_input = tfidf_input + "|" + " "
#tfidf_input

In [156]:
tfidf_input = tfidf_input.split("|")
tfidf_input.remove(' ')
tfidf_input = list(filter(None,tfidf_input))
tfidf_input[0][:2000]

'fit vocals bass time dont people throw sennisher cx promise sony xb55 nothing bass vocals piston earphone time days time lets earphone litter brother piston fit sound quality bass treble quality superb bro quality earphone quality rupees piston fit aluminum brand aluminum rupees price range trust cons cons price mic piston doesnt budget piston fit everything \n month earphone pay \n budget buyer thing youre review pay use mi phone use enhancer bass mi bass soo earphones hear anything music design buy quality price price bass lover use device product price range quality quality trendy service headphone months left speaker replacement product months product issues tests conclusion headphone headphone devices somehow use device time damage use android phones bass headphone queries buy product month purchase iam claim earphone budget category \n earphone gym purpose month boat bass equilizer output phone issue output price durability issue think twicejust product sound qualitytrust freind

In [157]:
from nltk.stem import WordNetLemmatizer 
lemmatizer = WordNetLemmatizer() 

temp_str = ''
temp_str = tfidf_input.copy()[0]
tfidf_input.clear()
temp_str_review = ''
tfidf_input = []
for word in temp_str.split(" "):
    #print(lemmatizer.lemmatize(word))
    new_word = lemmatizer.lemmatize(word)
    temp_str_review = temp_str_review+" "+new_word
tfidf_input.append(temp_str_review)
tfidf_input

[' fit vocal bass time dont people throw sennisher cx promise sony xb55 nothing bass vocal piston earphone time day time let earphone litter brother piston fit sound quality bass treble quality superb bro quality earphone quality rupee piston fit aluminum brand aluminum rupee price range trust con con price mic piston doesnt budget piston fit everything \n month earphone pay \n budget buyer thing youre review pay use mi phone use enhancer bass mi bass soo earphone hear anything music design buy quality price price bass lover use device product price range quality quality trendy service headphone month left speaker replacement product month product issue test conclusion headphone headphone device somehow use device time damage use android phone bass headphone query buy product month purchase iam claim earphone budget category \n earphone gym purpose month boat bass equilizer output phone issue output price durability issue think twicejust product sound qualitytrust freind earphone likes

In [252]:
from sklearn.feature_extraction.text import CountVectorizer
import re
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.feature_extraction.text import TfidfTransformer

corpus = tfidf_input
tf = TfidfVectorizer(analyzer='word', min_df = 0, stop_words = 'english')
tfidf_matrix =  tf.fit_transform(corpus)
feature_names = tf.get_feature_names()
#list(feature_names)[:50]

/usr/local/lib/python3.4/dist-packages/sklearn/feature_extraction/text.py:1028: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


In [253]:
def sort_coo(coo_matrix):
    tuples = zip(coo_matrix.col, coo_matrix.data)
    return sorted(tuples, key=lambda x: (x[1], x[0]), reverse=True)

def extract_topn_from_vector(feature_names, sorted_items, topn=10):
    """get the feature names and tf-idf score of top n items"""
    
    #use only topn items from vector
    sorted_items = sorted_items[:topn]

    score_vals = []
    feature_vals = []

    for idx, score in sorted_items:
        fname = feature_names[idx]
        
        #keep track of feature name and its corresponding score
        score_vals.append(round(score, 3))
        feature_vals.append(feature_names[idx])

    #create a tuples of feature,score
    #results = zip(feature_vals,score_vals)
    results= {}
    for idx in range(len(feature_vals)):
        results[feature_vals[idx]]=score_vals[idx]
    
    return results

unigram_dict = {}
sorted_items=sort_coo(tfidf_matrix.tocoo())
keywords=extract_topn_from_vector(feature_names,sorted_items,30)
for k in keywords:
    unigram_dict[k] = keywords[k]
    #print(k,keywords[k])
unigram_features = sorted(unigram_dict.items(), key=lambda kv: kv[1], reverse=True)
unigram_features

[('quality', 0.534),
 ('earphone', 0.523),
 ('price', 0.324),
 ('product', 0.272),
 ('sound', 0.253),
 ('bass', 0.181),
 ('month', 0.172),
 ('range', 0.1),
 ('ear', 0.097),
 ('headphone', 0.093),
 ('money', 0.091),
 ('phone', 0.086),
 ('music', 0.081),
 ('budget', 0.079),
 ('volume', 0.072),
 ('durability', 0.072),
 ('time', 0.068),
 ('use', 0.063),
 ('wire', 0.054),
 ('material', 0.05),
 ('cancellation', 0.05),
 ('mi', 0.05),
 ('day', 0.043),
 ('buy', 0.043),
 ('issue', 0.041),
 ('thing', 0.041),
 ('design', 0.041),
 ('brand', 0.036),
 ('cable', 0.036),
 ('song', 0.034)]

## Bigrams (me)

In [254]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

arr = tfidf_input #OR stuff

# If you want to take into account just term frequencies:
vectorizer = CountVectorizer(ngram_range=(2,2))
# The ngram range specifies your ngram configuration.

X = vectorizer.fit_transform(arr)
# Testing the ngram generation:
feature_names = vectorizer.get_feature_names()
print(feature_names[15:30])

['alley earphone', 'alot mat', 'alsoi headphone', 'alt return', 'alternative pair', 'aluminum brand', 'aluminum rupee', 'amazon delivery', 'ambiance feel', 'amount bass', 'amount music', 'amount noise', 'amzon judge', 'android phone', 'android user']


In [255]:
new_feature_names = []
for i in range(len(feature_names)):
    if feature_names[i] in bulk_review_lower_clean:
        new_feature_names.append(feature_names[i])
new_feature_names

['access google',
 'android phone',
 'android user',
 'apple earphone',
 'apple headphone',
 'audio enhancer',
 'audio output',
 'audio quality',
 'audio session',
 'audio work',
 'average earphone',
 'awasome earphone',
 'awesome earphone',
 'balance sound',
 'base effect',
 'bass advocate',
 'bass ear',
 'bass earphone',
 'bass effect',
 'bass enhancer',
 'bass head',
 'bass headphone',
 'bass level',
 'bass lover',
 'bass output',
 'bass quality',
 'bass response',
 'bass something',
 'bass sound',
 'bhi khatam',
 'boat bass',
 'boat earphone',
 'boat rocker',
 'boom headset',
 'boom type',
 'brand nowadays',
 'brand product',
 'bud size',
 'budget bt',
 'budget buyer',
 'budget category',
 'budget ear',
 'budget earphone',
 'budget headphone',
 'budget range',
 'build quality',
 'bush quality',
 'buy sony',
 'cable earphone',
 'cable look',
 'call clarity',
 'call shortcut',
 'cancellation wise',
 'cancellation work',
 'cannot use',
 'cause headache',
 'cause pain',
 'cheap earphon

In [256]:
bigram_dict = {}
features=[]
feature_count= []
for bigram in new_feature_names:
    bigram_dict[bigram] = stuff[0].count(bigram)
bigram_features = sorted(bigram_dict.items(), key=lambda kv: kv[1], reverse=True)
#print(bigram_features[:10])
bigram_features_top = bigram_features[:10]

for feature in bigram_features_top:
    features.append(feature[0])
print(features)

['sound quality', 'price range', 'mi earphone', 'quality good', 'build quality', 'noise cancellation', 'material quality', 'side ear', 'ear phone', 'mi phone']


In [257]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

arr = tfidf_input #OR stuff

# If you want to take into account just term frequencies:
vectorizer = CountVectorizer(ngram_range=(3,3))
# The ngram range specifies your ngram configuration.

X = vectorizer.fit_transform(arr)
# Testing the ngram generation:
feature_names = vectorizer.get_feature_names()
print(feature_names[15:30])

['alley earphone room', 'alot mat enhances', 'alsoi headphone verth', 'alt return buck', 'alternative pair budget', 'aluminum brand aluminum', 'aluminum rupee price', 'amazon delivery genuine', 'ambiance feel grade', 'amount bass please', 'amount music quality', 'amount noise con', 'amzon judge box', 'android phone bass', 'android user earphone']


In [258]:
new_feature_names = []
for i in range(len(feature_names)):
    if feature_names[i] in bulk_review_lower_clean:
        new_feature_names.append(feature_names[i])
new_feature_names

['access google assistant',
 'balance sound signature',
 'boat earphone boat',
 'boom type headset',
 'budget ear phone',
 'decent sound quality',
 'dip bass something',
 'dont waste money',
 'drop silent office',
 'ear phone design',
 'ear sound quality',
 'grey color earphone',
 'loud music song',
 'mi service centre',
 'money volume level',
 'noise cancellation work',
 'product get problem',
 'qr xode scanner',
 'quality sound quality',
 'quality till date',
 'shout loud enogh',
 'side speaker doesnt',
 'size ear plug',
 'sound quality bass',
 'sound quality mi',
 'sound quality spot',
 'sound quality wuth',
 'stock android user',
 'test sound track',
 'throw sennisher cx',
 'use mi phone']

In [259]:
trigram_dict = {}
#features=[]
feature_count= []
for trigram in new_feature_names:
    trigram_dict[trigram] = stuff[0].count(trigram)
trigram_features = sorted(trigram_dict.items(), key=lambda kv: kv[1], reverse=True)
#print(bigram_features[:10])
trigram_feautures_top = trigram_features[:1]

for feature in trigram_feautures_top:
    features.append(feature[0])
print(features)

['sound quality', 'price range', 'mi earphone', 'quality good', 'build quality', 'noise cancellation', 'material quality', 'side ear', 'ear phone', 'mi phone', 'sound quality bass']


In [260]:
from gensim.scripts.glove2word2vec import glove2word2vec

'''glove_input_file = 'glove.6B.50d.txt'
word2vec_output_file = 'glove.6B.100d.txt.word2vec'
glove2word2vec(glove_input_file, word2vec_output_file)'''

"glove_input_file = 'glove.6B.50d.txt'\nword2vec_output_file = 'glove.6B.100d.txt.word2vec'\nglove2word2vec(glove_input_file, word2vec_output_file)"

In [25]:
from gensim.models import KeyedVectors
#load the Stanford GloVe model
filename = 'glove.6B.100d.txt.word2vec'
model = KeyedVectors.load_word2vec_format(filename, binary=False)
# calculate: (king - man) + woman = ?
#result = model.most_similar(positive=['woman', 'king'], negative=['man'], topn=1)
#result= model.most_similar('battery')
model.wv.save_word2vec_format('model.bin')

/usr/local/lib/python3.4/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  


In [261]:
features_uni={}
for feat in unigram_features:
    tup=feat[1] + model.similarity('phone',feat[0])*0
    features_uni[feat[0]]=tup
    features_temp = sorted(features_uni.items(), key=lambda kv: kv[1], reverse=True)

for i in features_temp[:20]:
    features.append(i[0])
features

['sound quality',
 'price range',
 'mi earphone',
 'quality good',
 'build quality',
 'noise cancellation',
 'material quality',
 'side ear',
 'ear phone',
 'mi phone',
 'sound quality bass',
 'quality',
 'earphone',
 'price',
 'product',
 'sound',
 'bass',
 'month',
 'range',
 'ear',
 'headphone',
 'money',
 'phone',
 'music',
 'budget',
 'volume',
 'durability',
 'time',
 'use',
 'wire',
 'material']

## Product related vocabulary

In [262]:
result= model.most_similar("camera")
print(result)

[('cameras', 0.8814679384231567), ('screen', 0.8745492100715637), ('screens', 0.7714206576347351), ('video', 0.7448007464408875), ('microphone', 0.7421790957450867), ('pictures', 0.7322328686714172), ('images', 0.7311602830886841), ('mirror', 0.7270058393478394), ('projector', 0.7233645915985107), ('device', 0.7233129143714905)]


## remove similar features

In [263]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
remove_features=[]
for i in range(len(features)):
    for j in range(i+1,len(features)):
        if(fuzz.ratio(features[i], features[j])> 60):
            print(features[i], features[j])
            if(len(features[i]) > len(features[j])):
                remove_features.append(features[j])
            else:
                remove_features.append(features[i])
#print(features)
#print(set(remove_features))
for i in range(len(features)):
    for j in range(i+1,len(features)):
        if(features[i] in features[j] or features[j] in features[i]):
            print(features[i], features[j])
            if(len(features[i]) > len(features[j])):
                remove_features.append(features[j])
            else:
                remove_features.append(features[i])

sound quality build quality
sound quality sound quality bass
sound quality quality
price range price
price range range
mi earphone ear phone
mi earphone mi phone
mi earphone earphone
mi earphone headphone
mi earphone phone
quality good quality
build quality material quality
build quality sound quality bass
build quality quality
material quality quality
material quality material
ear phone mi phone
ear phone earphone
ear phone headphone
ear phone phone
mi phone earphone
mi phone money
mi phone phone
quality durability
earphone headphone
earphone phone
headphone phone
sound quality sound quality bass
sound quality quality
sound quality sound
price range price
price range range
mi earphone earphone
mi earphone ear
mi earphone phone
quality good quality
build quality quality
material quality quality
material quality material
side ear ear
ear phone ear
ear phone phone
mi phone phone
sound quality bass quality
sound quality bass sound
sound quality bass bass
earphone ear
earphone phone
headph

In [264]:
#print(remove_features)
for i in set(remove_features):
    features.remove(i)
features

['price range',
 'mi earphone',
 'quality good',
 'noise cancellation',
 'material quality',
 'side ear',
 'sound quality bass',
 'product',
 'month',
 'music',
 'budget',
 'volume',
 'durability',
 'time',
 'use',
 'wire']

## Sentence Extraction

In [ ]:
import spacy
nlp = spacy.load('en')
from spacy import displacy
parsed_bulk_review = nlp(bulk_review_lower_clean)
parsed_bulk_review[:200]

In [170]:
spacy_sentences = []
for num, sentence in enumerate(parsed_bulk_review.sents):
    spacy_sentences.append(sentence)
spacy_sentences

[bang for bucks 
          read my review  earphones i own sennisher cx 180 mi in-ear,
 basic 1 more piston fit sony xb-55 and am a audiophile i like vocals and deep bass at the same time cx,
 180 is the worst (i do not know why people  are mad about cx 180 but after using mi in ear basic u will throw sennisher,
 cx 180,
 i promise that) sony xb55 has nothing but bass ( no vocals and treble) 1 more piston is the best under rs.,
 1000 earphone,
 (give it a time 20-30 days driver burn time) now lets talk about mi in ear,
 basic earphone,
 i would say mi in ear is litter brother of  1more piston fit.  ,
 its sound quality is superb .,
 it has  deep bass  and good treble.,
 i would say  its all rounder.,
 mic quality is superb  bro i recorded audio from its mic and was blown away .they are providing  best sound and mic quality in under 400 earphone .,
 build quality is like you are holding 1000 rupees earphone ever better than piston fit .,
 it has aluminum casing  .,
 no brand is providin

'''
format:

step1 = {
    "feature":
    {
        "sentiment":
        {
        ""
        }
        "sentences":
        {
        "",
        "",
        "",
        ...
        }
        "adjectives":
        "",
        "",
        "",
        ...
    }
}
'''

In [210]:
step1_dict = dict()
for feature in features:
    step1_dict[feature] = dict()
step2_dict = step1_dict.copy()
step1_dict

{'bass': {},
 'budget': {},
 'durability': {},
 'material quality': {},
 'mi earphone': {},
 'month': {},
 'music': {},
 'noise cancellation': {},
 'price range': {},
 'product': {},
 'side ear': {},
 'sound quality good': {},
 'time': {},
 'use': {},
 'value money': {},
 'volume': {},
 'wire': {}}

In [211]:
feature_sentences = []
feature_adjectives = []

for feature in features:
    for spacy_sentence in spacy_sentences:
        if feature in str(spacy_sentence):
            feature_sentences.append(str(spacy_sentence))
    ### BIGRAMS WITHOUT STOPWORDS____ SOLUTION
    if len(feature_sentences)==0:
        for spacy_sentence in spacy_sentences:
            if feature.split(' ')[0] in str(spacy_sentence):
                feature_sentences.append(str(spacy_sentence))
    step1_dict[feature]['sentences'] =  feature_sentences[:]
    feature_sentences.clear()

print(step1_dict)

{'wire': {'sentences': ['score-4/5 wire is not flat or tangle free, though it seems it would not tangle.', 'wire does not feel cheap like most cheap earphones.', 'decent choice for decent budget \n         there is nothing to be hyped about this earphone, they are simple with decent audio quality comes with wire which gets tangled go for this', 'wires and ear buds are also does not look very strong.', 'subsequent cutting ad a standard 3.5 mm pin whose wires smoothly blend into each connection without much sharp joints.', '- secondly the worst thing i feel is that a have stuck a stupid warranty tag on the wires of the earphones which says warranty void if the tag is removed.', 'this is absolutely stupid because who like a stupid piece of small paper hanging around the wires for a year just to claim warranty.', 'then at 3rd month its left earbud stopped producing sound tip:- if u want to check that ur product is genuine then scan the qr code tied on earphone wire near the 3.5 mm jack wit

In [212]:
sample_adjectives=[]
for feature in features:
    for spacy_sentence in spacy_sentences:
        if feature in str(spacy_sentence):
            dependencies = [(x.text, x.pos_,x.dep_) for x in spacy_sentence]
            #print(dependencies)
            search_noun = feature.split(" ")[0]
            adjective=""
            adjective2=""
            #sample_adjectives=[]
            for i in range(len(dependencies)):
                if (dependencies[i][0]== search_noun):
                    j=i
                    k=i
                    dist1=0
                    dist2=0
                    flag1=0
                    while(j>=0):
                        '''
                        if(dependencies[j][1]=='ADV'):
                            adjective = adjective + dependencies[j][1]
                            '''
                        if(dependencies[j][1]=='ADJ' and (dependencies[j][2]=='amod' or dependencies[j][2]=='conj')):
                            adjective += " " + dependencies[j][0]
                            flag1=1
                        if(dependencies[j-1][1]=='ADV'):
                            #print("adv:",dependencies[j-1][0] )
                            adjective = dependencies[j-1][0] + " " + adjective
                            break;
                        elif(dependencies[j-1][1]!='ADV' and flag1==1):
                            break;
                        j=j-1
                        dist1= dist1+1
                    if(flag1==0):
                        dist1=10000

                    while(k<len(dependencies)):
                        if(dependencies[k][1]=='ADV'):
                            adjective2 = adjective2 + dependencies[k][0] + " "
                        if(dependencies[k][1]=='ADJ' and (dependencies[k][2]=='acomp' or dependencies[k][2]=='attr' or dependencies[k][2]=='advmod' )):
                            if(dist2 < dist1):
                                adjective2 += " " + dependencies[k][0]
                                adjective= adjective2
                            break;
                        k=k+1
                        dist2= dist2+1
                    if adjective!='':
                        sample_adjectives.append(adjective)
                    adjective=""
                    adjective1=""

    #print("feature=",feature,"adjectives=", sample_adjectives)
    step1_dict[feature]['adjectives'] = sample_adjectives[:]
    sample_adjectives.clear()
    
#print(step1_dict['battery life'])

In [218]:
for feature in features:
    print(feature,": ",step1_dict[feature]['adjectives'])

price range :  ['absolutely  cool', ' best', ' best', 'very  good', 'very ', ' other', ' sound', ' other', ' good', ' other', ' better', 'very ', 'nt ', ' nice', 'very ', ' awesome', ' awesome', ' amazing', ' good', ' cheaper', ' great', ' awesome', 'very  good', ' okay', ' cheap', ' best', ' best', 'best ', ' best', 'quite ']
mi earphone :  [' new', 'surprisingly more  comfortable', 'when ', ' original', ' higher', ' black', 'also ', 'not ', ' same', 'also ', 'back ', ' awesome', 'then ', ' perfect', 'when ', ' fake', ' original', ' original', ' original', ' such', 'not ', ' unbelievable', 'now ']
noise cancellation :  [' good', 'when ', ' sound', 'not  sure', ' awesome', ' sound', 'when ', 'not  excellent', ' great', ' great', ' decent', ' better', ' much', ' amazing', ' amazing', ' good', 'not very  effective']
value money :  []
material quality :  [' good', ' overall', ' material', ' worthless', ' decent', ' satisfied', ' material', ' material', ' material', ' material', ' awesome'

## Sentiment

#### IBM WATSON

In [217]:
import json
from watson_developer_cloud import NaturalLanguageUnderstandingV1
from watson_developer_cloud.natural_language_understanding_v1 import Features, SentimentOptions
print(len(features))
def find_sentiment(features):
    try:
        natural_language_understanding = NaturalLanguageUnderstandingV1(
            version='2018-11-16',
            iam_apikey='gm6iXTWNo8uw12AcIx4LVjkNt39APWHgFS1py2w4Zzki',
            url='https://gateway-wdc.watsonplatform.net/natural-language-understanding/api'
        )

        response = natural_language_understanding.analyze(
            html="<html>"+ "<p>" + bulk_review_lower_clean + "</p>" + "</html>",
            features=Features(sentiment=SentimentOptions(targets=features))).get_result()

        return(response['sentiment']) ##to be written on a json file
    except requests.exceptions.ConnectionError:
        return("Connection refused")
        
sentiment_dict = find_sentiment(features)
len(sentiment_dict['targets'])

17


14

In [209]:
for entity in sentiment_dict['targets']:
    step1_dict[entity['text']]['sentiment'] = entity['score']
    #print(entity['text'], step1_dict[entity['text']]['sentiment'] )

for feature in features:
    print(feature,": ",step1_dict[feature])

price range :  {'adjectives': ['absolutely  cool', ' best', ' best', 'very  good', 'very ', ' other', ' sound', ' other', ' good', ' other', ' better', 'very ', 'nt ', ' nice', 'very ', ' awesome', ' awesome', ' amazing', ' good', ' cheaper', ' great', ' awesome', 'very  good', ' okay', ' cheap', ' best', ' best', 'best ', ' best', 'quite '], 'sentences': ['no brand is providing aluminum casing under 400 rupees  earphone .. and if u are a youtuber its mic can replace mic under price range of 1000 ..it has a very good mic  trust me cons :-  ', 'absolutely cool product considering the price range.', 'one of the best earphones in the price range.', 'and i was surprised that how good these were, good sound,  awesome noise cancellation and plus you get mike function and best part is price range.', 'these earphones looks very good in ears - very comfortable after listening music for 1hour no pain in ears very comfortable - sound is clear and nice - provided button is helpful can pause song ,

## Output :

In [184]:
from textblob import TextBlob
import statistics
import numpy as np

final_output = dict()

for feature in features:
    word_polarity=[]
    for adj in step1_dict[feature]['adjectives']:
        word_polarity.append((TextBlob(adj).sentiment)[0])
    if len(word_polarity)!=0:
        word_adjectives = list(zip(sample_adjectives,word_polarity))
        #print(word_polarity)
        average_sentiment= statistics.mean(word_polarity)  

        centralised_polarity= []
        centralised_polarity[:] = [abs(average_sentiment - x) for x in word_polarity]
        ind = np.argmin(centralised_polarity)
        final_adjective = step1_dict[feature]['adjectives'][ind]
        #step2_dict[feature]['adjective'] = final_adjective
        #print("->",feature,"\t->",final_adjective)
        final_output[feature] = final_adjective

#step2_dict
final_output

{'bass': ' full',
 'budget': 'daily  great',
 'durability': 'not ',
 'material quality': ' decent',
 'mi earphone': ' new',
 'month': ' basic',
 'music': 'very ',
 'noise cancellation': ' sound',
 'price range': ' nice',
 'product': 'very much  comparable',
 'side ear': ' right',
 'time': ' same',
 'use': 'somehow ',
 'volume': ' high',
 'wire': 'not very  careful'}